In [1]:
import random
import time
from IPython.display import clear_output

In [2]:
def combinacoes_de_dados(lista):
    combinacoes = set()

    combinacoes.add((lista[0] + lista[1],lista[2] + lista[3]))
    combinacoes.add((lista[0] + lista[2],lista[1] + lista[3]))
    combinacoes.add((lista[0] + lista[3],lista[1] + lista[2]))

    return list(combinacoes)


In [3]:
class Column():
    def __init__(self,length):
        self.length=length
        self.position={1:0,2:0}
        self.marker=0
        self.blocked=False

    def advance_marker(self):
        self.marker+=1

    def record_advance(self,active_player):
        self.position[active_player] = self.marker
        self.marker_reset()

    def marker_reset(self):
        self.marker = 0

    def marker_start(self,active_player):
        self.marker = self.position[active_player] + 1

    def possui_marcador(self):
        return self.marker > 0
    
    def block_column(self):
        self.blocked=True
    
    def is_blocked(self):
        return self.blocked

In [4]:
n_colunas = [2,4,6,8,10,12,10,8,6,4,2]
class Board():
    def __init__(self):
        self.columns={}
        for i in range(11):
            self.columns[i+2] = Column(n_colunas[i])
    
    def roll_dice(self):
        return combinacoes_de_dados([random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7)])

    def lista_colunas_com_marcadores(self):
        return list(filter( lambda coluna : self.columns[coluna].possui_marcador(), self.columns))
    
    def lista_colunas_bloqueadas(self):
        return list(filter( lambda coluna : self.columns[coluna].is_blocked(), self.columns))
       

In [5]:
class Game_settings():
    def __init__(self):
        self.marker_count=3
    
    def set_rules(self):
        global marker_count
        marker_count=self.marker_count

In [6]:
class Player():
    def __init__(self,name):
        player.name=self.name


In [7]:
game_settings=Game_settings()
game_settings.set_rules()

In [8]:
def run_game():
    #initialize the classes
    board=Board()
    active_player=1
    points=[]
    #choose your move
    def make_move(your_choice):
        '''execute the chosen move'''
        #make your move
        made_move=False
        while(your_choice!=[]):
            for i in range (2,13):
                if your_choice==[]:
                    break
                elif ((your_choice[0] not in board.lista_colunas_com_marcadores()) 
                      and len(board.lista_colunas_com_marcadores())<marker_count
                      and your_choice[0] not in board.lista_colunas_bloqueadas()): #se tem marcardor disponível, bota marcador

                    board.columns[your_choice[0]].marker_start(active_player)
                    print(f'placed marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])
                    made_move=True

                elif your_choice[0] in board.lista_colunas_com_marcadores(): #se tem marcador posicionado, avança marcador

                    board.columns[your_choice[0]].advance_marker()
                    print(f'advanced marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])
                    made_move=True
                
                else:
                    print('')
                    print('')
                    your_choice.remove(your_choice[0])
                    
        if made_move==False:
            for i in range(2,13):
                board.columns[i].marker_reset()
                print('no possible moves, markers were reset')
                return (False)
        return True
                    
    def pass_turn(active_player):
        '''voluntarily end your turn'''
        for i in range (2,13):
            if i in board.lista_colunas_com_marcadores():
                board.columns[i].record_advance(active_player)
                if board.columns[i].position[active_player]>=board.columns[i].length:
                    board.columns[i].block_column()
                    points.append(active_player)

            
    for i in range(500):
        
        time.sleep(0.1)
        clear_output(wait=True)
        your_dice=board.roll_dice()
        your_choice=list(your_dice[random.randrange(0,len(your_dice))])
        print(f'player {active_player} has chosen: {your_choice}')
        if make_move(your_choice):
            if active_player==1:
                active_player=2
            else:
                active_player=1            
        if len(board.lista_colunas_com_marcadores())>=3:
            pass_turn(active_player)
            if active_player==1:
                active_player=2
            else:
                active_player=1
        print(f'Game Board: \nColumns with markers: {board.lista_colunas_com_marcadores()}')
        print(f'Blocked columns: {board.lista_colunas_bloqueadas()}')
        print(f'Points:: {points}')
        for i in range (2,13):
            print(f'column {i}: {board.columns[i].position}')
        if len(points)>=5:
            break

In [9]:
run_game()

player 1 has chosen: [5, 5]
placed marker at column: 5
5 7
advanced marker at column: 5
5 8
Game Board: 
Columns with markers: []
Blocked columns: [2, 4, 5, 6, 10]
Points:: [2, 2, 2, 2, 2]
column 2: {1: 1, 2: 2}
column 3: {1: 3, 2: 3}
column 4: {1: 3, 2: 6}
column 5: {1: 6, 2: 8}
column 6: {1: 7, 2: 10}
column 7: {1: 5, 2: 3}
column 8: {1: 7, 2: 7}
column 9: {1: 3, 2: 4}
column 10: {1: 4, 2: 6}
column 11: {1: 2, 2: 2}
column 12: {1: 1, 2: 0}
